In [1]:
import pybullet as p
import time
import pybullet_data
physicsClient = p.connect(p.GUI)#or p.DIRECT for non-graphical version
p.setAdditionalSearchPath(pybullet_data.getDataPath()) #optionally

In [2]:
planeId = p.loadURDF("plane.urdf")

In [3]:
startPos = [0,0,0]
startOrientation = p.getQuaternionFromEuler([0,0,0])
robotId = p.loadURDF("../ik/robot.urdf", startPos, startOrientation)

In [4]:
import threading
def sim_main():
    while True:
        p.stepSimulation()
        time.sleep(1./240.)
sim_t = threading.Thread(target=sim_main)
sim_t.start()

↑ ここまでpybulletの初期化 (1回しか実行しちゃいけない類のやつ)

----

In [23]:
import webcface
wcli = webcface.Client("bullet-sim")
wcli.start()

In [71]:
# dynamixelに仕様をあわせる
import math
@wcli.func()
def move_servo(angle1: float, angle2: float, angle3: float, angle4: float, angle5: float, angle6: float, angle7: float):
    arm_joint = [30, 31, 33, 35, 36, 37, 39]
    p.setJointMotorControl2(robotId, arm_joint[0], p.POSITION_CONTROL, math.radians(angle1 / 1023 * 300 - 150), force=1)
    p.setJointMotorControl2(robotId, arm_joint[1], p.POSITION_CONTROL, math.radians(angle2 / 1023 * 300 - 150), force=1)
    p.setJointMotorControl2(robotId, arm_joint[2], p.POSITION_CONTROL, math.radians(angle3 / 1023 * 300 - 150), force=1)
    p.setJointMotorControl2(robotId, arm_joint[3], p.POSITION_CONTROL, math.radians(angle4 / 1023 * 300 - 150), force=1)
    p.setJointMotorControl2(robotId, arm_joint[4], p.POSITION_CONTROL, math.radians(angle5 / 1023 * 300 - 150), force=1)
    p.setJointMotorControl2(robotId, arm_joint[5], p.POSITION_CONTROL, math.radians(angle6 / 1023 * 300 - 150), force=1)
    p.setJointMotorControl2(robotId, arm_joint[6], p.POSITION_CONTROL, math.radians(angle7 / 1023 * 300 - 150), force=1)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "C:\Python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\msys64\tmp\ipykernel_12692\2076629836.py", line 4, in sim_main
pybullet.error: Not connected to physics server.


----

In [20]:
p.setGravity(0,0,-10)

In [68]:
p.resetBasePositionAndOrientation(robotId, startPos, startOrientation)
p.resetBaseVelocity(robotId, [0, 0, 0], [0, 0, 0])

In [5]:
p.getNumJoints(robotId)

47

In [6]:
for i in range(p.getNumJoints(robotId)):
    print(i, p.getJointInfo(robotId, i))

0 (0, b'base_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'base_link', (0.0, 0.0, 0.0), (0.0, 0.0, 0.0102), (0.0, 0.0, 0.0, 1.0), -1)
1 (1, b'wheel_left_joint', 0, 7, 6, 1, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'wheel_left_link', (0.0, 0.0, 1.0), (-0.01, 0.115, 0.025), (0.7071067811865475, 0.0, 0.0, 0.7071067811865476), 0)
2 (2, b'wheel_right_joint', 0, 8, 7, 1, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'wheel_right_link', (0.0, 0.0, 1.0), (-0.01, -0.115, 0.025), (0.7071067811865475, 0.0, 0.0, 0.7071067811865476), 0)
3 (3, b'caster_front_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'caster_front_link', (0.0, 0.0, 0.0), (0.10500000000000001, 0.0, 0.007), (0.7071067811865475, 0.0, 0.0, 0.7071067811865476), 0)
4 (4, b'caster_back_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'caster_back_link', (0.0, 0.0, 0.0), (-0.14500000000000002, 0.0, 0.009), (0.7071067811865475, 0.0, 0.0, 0.7071067811865476), 0)
5 (5, b'gyro_joint', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'gyro_link',

In [7]:
arm_joint = [30, 31, 33, 35, 36, 37, 39]

In [16]:
p.setJointMotorControl2(
    robotId,
    arm_joint[0],
    controlMode=p.POSITION_CONTROL,
    targetPosition=0
)
p.setJointMotorControl2(
    robotId,
    arm_joint[1],
    controlMode=p.POSITION_CONTROL,
    targetPosition=1
)
p.setJointMotorControl2(
    robotId,
    arm_joint[2],
    controlMode=p.POSITION_CONTROL,
    targetPosition=1
)

In [67]:
for i in range(7):
    p.setJointMotorControl2(
        robotId,
        arm_joint[i],
        controlMode=p.POSITION_CONTROL,
        targetPosition=0,
        force=10
    )

In [70]:
# 1個ずつ1rad→0radに動かす
for i in range(7):
    p.setJointMotorControl2(
        robotId,
        arm_joint[i],
        controlMode=p.POSITION_CONTROL,
        targetPosition=1,
        force=1
    )
    time.sleep(2)
    p.setJointMotorControl2(
        robotId,
        arm_joint[i],
        controlMode=p.POSITION_CONTROL,
        targetPosition=0,
        force=1
    )
    time.sleep(2)

KeyboardInterrupt: 